## Model : Vgg16

----------train-----------

In [9]:
cd /content/drive/MyDrive/Colab Notebooks/cat_dogs

/content/drive/MyDrive/Colab Notebooks/cat_dogs


In [4]:
!unzip cat_dogs.zip

Archive:  cat_dogs.zip
   creating: test/
   creating: test/cat/
  inflating: test/cat/cat.12250.jpg  
  inflating: test/cat/cat.12251.jpg  
  inflating: test/cat/cat.12252.jpg  
  inflating: test/cat/cat.12253.jpg  
  inflating: test/cat/cat.12254.jpg  
  inflating: test/cat/cat.12255.jpg  
  inflating: test/cat/cat.12256.jpg  
  inflating: test/cat/cat.12257.jpg  
  inflating: test/cat/cat.12258.jpg  
  inflating: test/cat/cat.12259.jpg  
  inflating: test/cat/cat.12260.jpg  
  inflating: test/cat/cat.12261.jpg  
  inflating: test/cat/cat.12262.jpg  
  inflating: test/cat/cat.12263.jpg  
  inflating: test/cat/cat.12264.jpg  
  inflating: test/cat/cat.12265.jpg  
  inflating: test/cat/cat.12266.jpg  
  inflating: test/cat/cat.12267.jpg  
  inflating: test/cat/cat.12268.jpg  
  inflating: test/cat/cat.12269.jpg  
  inflating: test/cat/cat.12270.jpg  
  inflating: test/cat/cat.12271.jpg  
  inflating: test/cat/cat.12272.jpg  
  inflating: test/cat/cat.12273.jpg  
  inflating: test/cat/c

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model

In [14]:
train = ImageDataGenerator().flow_from_directory('train',
                                                 target_size=(224,224),
                                                 batch_size=10)
valid = ImageDataGenerator().flow_from_directory('valid',
                                                 target_size=(224,224),
                                                 batch_size=30)


Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 2 classes.


In [15]:
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg') # pooling은 global의 의미

58889256/58889256 [==============================] - 3s 0us/step


In [16]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
for layer in base_model.layers[:-5] :
  layer.trainable = False

In [18]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
last_output = base_model.output

x = Dense(2, activation='softmax')(last_output)


In [20]:
new_model = Model(inputs=base_model.input, outputs =x)

In [21]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [22]:
from keras.optimizers import Adam

In [23]:
new_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',
                  metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [24]:
from keras.callbacks import ModelCheckpoint

In [25]:
checkpointer = ModelCheckpoint(filepath='cat_dogs.model.hdf5',
                               save_best_only=True)

In [26]:
hist = new_model.fit_generator(train, epochs=20, validation_data=valid, verbose=1, callbacks=[checkpointer])

Epoch 1/20


<ipython-input-26-1291c1032fb9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = new_model.fit_generator(train, epochs=20, validation_data=valid, verbose=1, callbacks=[checkpointer])


21/21 [==============================] - 17s 348ms/step - loss: 1.3193 - accuracy: 0.7673 - val_loss: 0.2315 - val_accuracy: 0.9126
Epoch 2/20
21/21 [==============================] - 2s 117ms/step - loss: 0.0553 - accuracy: 0.9802 - val_loss: 0.1806 - val_accuracy: 0.9515
Epoch 3/20
21/21 [==============================] - 2s 118ms/step - loss: 0.0172 - accuracy: 0.9950 - val_loss: 0.1799 - val_accuracy: 0.9320
Epoch 4/20
21/21 [==============================] - 2s 100ms/step - loss: 0.0077 - accuracy: 0.9950 - val_loss: 0.2018 - val_accuracy: 0.9029
Epoch 5/20
21/21 [==============================] - 2s 117ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.1409 - val_accuracy: 0.9320
Epoch 6/20
21/21 [==============================] - 2s 99ms/step - loss: 8.9494e-04 - accuracy: 1.0000 - val_loss: 0.1526 - val_accuracy: 0.9126
Epoch 7/20
21/21 [==============================] - 2s 100ms/step - loss: 4.4697e-04 - accuracy: 1.0000 - val_loss: 0.1708 - val_accuracy: 0.9029
Epoch 8/2

--------------test-----------

In [27]:
from sklearn.datasets import load_files
import numpy as np
from keras.utils.np_utils import to_categorical #one-hot encoding

In [28]:
data = load_files('test')

In [29]:
data['filenames']

array(['test/cat/cat.12374.jpg', 'test/cat/cat.12304.jpg',
       'test/dog/dog.12018.jpg', 'test/dog/dog.12043.jpg',
       'test/cat/cat.12480.jpg', 'test/cat/cat.12384.jpg',
       'test/cat/cat.12262.jpg', 'test/dog/dog.12111.jpg',
       'test/dog/dog.12022.jpg', 'test/cat/cat.12326.jpg',
       'test/dog/dog.12130.jpg', 'test/cat/cat.12407.jpg',
       'test/dog/dog.12072.jpg', 'test/cat/cat.12414.jpg',
       'test/dog/dog.12100.jpg', 'test/cat/cat.12271.jpg',
       'test/cat/cat.12368.jpg', 'test/cat/cat.12251.jpg',
       'test/cat/cat.12260.jpg', 'test/dog/dog.12002.jpg',
       'test/dog/dog.12116.jpg', 'test/cat/cat.12429.jpg',
       'test/dog/dog.12170.jpg', 'test/cat/cat.12363.jpg',
       'test/dog/dog.12045.jpg', 'test/dog/dog.12141.jpg',
       'test/dog/dog.12096.jpg', 'test/cat/cat.12470.jpg',
       'test/cat/cat.12474.jpg', 'test/cat/cat.12463.jpg',
       'test/dog/dog.12194.jpg', 'test/dog/dog.12127.jpg',
       'test/dog/dog.12060.jpg', 'test/cat/cat.12287.jpg

In [30]:
paths = np.array(data['filenames'])
targets = to_categorical(data['target'])

In [31]:
test_files, test_target = data['filenames'], targets

In [32]:
from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

In [33]:
def path_to_tensor(img_path):
  img= load_img(img_path, target_size=(224,224))
  x = img_to_array(img)
  # return x[np.newaxis, :]
  return np.expand_dims(x, axis=0)

In [34]:
def paths_to_tensor(img_paths):
  list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
  return np.vstack(list_of_tensors)

In [35]:
test_tensor = paths_to_tensor(test_files)

In [36]:
test_tensor=preprocess_input(test_tensor) #vgg16에 맞게 데이터를 preprocessing

In [40]:
new_model.evaluate(test_tensor, test_target)

# test 의 loss, accuracy

15/15 [==============================] - 2s 138ms/step - loss: 0.3286 - accuracy: 0.9268


[0.32862311601638794, 0.9268292784690857]

-----

-------실제 예측-------

In [41]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/dog-1203760_960_720.jpg', target_size=(224,224))

In [50]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/dog.jpg', target_size=(224,224))

In [56]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/pred001.jfif', target_size=(224,224))

In [68]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/_126241775_getty_cats.png', target_size=(224,224))

In [74]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/cat.jfif', target_size=(224,224))

In [62]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/main-ad65ae47-5a50-456d-a41f-528b63071b7b.jpg', target_size=(224,224))

JPG 용

In [86]:
num = ['0562', '1311', '1459', '1464', '2229', '2232','0914','0915']
for i in num :
  img = load_img(f'/content/drive/MyDrive/Colab Notebooks/cat_dogs/IMG_{i}.jpg', target_size=(224,224))

  img = img_to_array(img)
  img = img.reshape((1, img.shape[0],img.shape[1],img.shape[2]))
  img = preprocess_input(img)
  pred = new_model.predict(img)
  if np.argmax(pred) == 1:
    print('i', '이거슨 \n개!')
  else :
    print('i', '이거슨 \n고양이!')

1/1 [==============================] - 0s 16ms/step
i 이거슨 
개!
1/1 [==============================] - 0s 19ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 19ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
개!
1/1 [==============================] - 0s 17ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
개!
1/1 [==============================] - 0s 22ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
개!


JPEG용

In [87]:
num = ['0916','0917','0918']
for i in num :
  img = load_img(f'/content/drive/MyDrive/Colab Notebooks/cat_dogs/IMG_{i}.jpeg', target_size=(224,224))

  img = img_to_array(img)
  img = img.reshape((1, img.shape[0],img.shape[1],img.shape[2]))
  img = preprocess_input(img)
  pred = new_model.predict(img)
  if np.argmax(pred) == 1:
    print('i', '이거슨 \n개!')
  else :
    print('i', '이거슨 \n고양이!')

1/1 [==============================] - 0s 19ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 18ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
고양이!


In [75]:
img = img_to_array(img)

In [76]:
img = img.reshape((1, img.shape[0],img.shape[1],img.shape[2]))

In [77]:
img = preprocess_input(img)

In [78]:
pred = new_model.predict(img)

1/1 [==============================] - 0s 17ms/step


In [79]:
if np.argmax(pred) == 1:
  print('이거슨 \n\n개!')
else :
  print('이거슨 \n\n고양이!')

이거슨 

고양이!


----

## Model : Xception

---------train----------

In [90]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [91]:

IMG_WIDTH = 224
IMG_HEIGHT = 224

In [92]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH,IMG_HEIGHT,3))

base_model.summary()

83683744/83683744 [==============================] - 4s 0us/step
Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                   

In [93]:
model = Sequential()

model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 16)                32784     
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 2)                 34        
                                                                 
Total params: 20,894,298
Trainable params: 20,839,770
Non-trainable params: 54,528
_______________________________________

In [95]:
train = ImageDataGenerator().flow_from_directory('train',
                                                 target_size=(224,224),
                                                 batch_size=10)
valid = ImageDataGenerator().flow_from_directory('valid',
                                                 target_size=(224,224),
                                                 batch_size=30)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.


In [96]:
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(2e-5), metrics=['accuracy'])

In [97]:
checkpointer = ModelCheckpoint(filepath='cat_dogs.model.hdf5',
                               save_best_only=True)

In [120]:
hist = model.fit_generator(train, epochs=200, validation_data=valid, verbose=1, callbacks=[checkpointer])

Epoch 1/200


<ipython-input-120-97fc1ff7863d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(train, epochs=200, validation_data=valid, verbose=1, callbacks=[checkpointer])


21/21 [==============================] - 8s 368ms/step - loss: 0.0336 - accuracy: 0.9901 - val_loss: 0.0980 - val_accuracy: 0.9612
Epoch 2/200
21/21 [==============================] - 6s 276ms/step - loss: 0.0555 - accuracy: 0.9901 - val_loss: 0.0973 - val_accuracy: 0.9612
Epoch 3/200
21/21 [==============================] - 5s 212ms/step - loss: 0.0356 - accuracy: 1.0000 - val_loss: 0.1004 - val_accuracy: 0.9515
Epoch 4/200
21/21 [==============================] - 6s 276ms/step - loss: 0.0394 - accuracy: 0.9950 - val_loss: 0.0957 - val_accuracy: 0.9612
Epoch 5/200
21/21 [==============================] - 6s 277ms/step - loss: 0.0255 - accuracy: 1.0000 - val_loss: 0.0904 - val_accuracy: 0.9612
Epoch 6/200
21/21 [==============================] - 6s 279ms/step - loss: 0.0322 - accuracy: 0.9950 - val_loss: 0.0891 - val_accuracy: 0.9709
Epoch 7/200
21/21 [==============================] - 6s 280ms/step - loss: 0.0625 - accuracy: 0.9802 - val_loss: 0.0828 - val_accuracy: 0.9612
Epoch 8/200

KeyboardInterrupt: ignored

--------------test-----------

In [99]:
from sklearn.datasets import load_files
import numpy as np
from keras.utils.np_utils import to_categorical #one-hot encoding

In [100]:
data = load_files('test')

In [101]:
data['filenames']

array(['test/cat/cat.12374.jpg', 'test/cat/cat.12304.jpg',
       'test/dog/dog.12018.jpg', 'test/dog/dog.12043.jpg',
       'test/cat/cat.12480.jpg', 'test/cat/cat.12384.jpg',
       'test/cat/cat.12262.jpg', 'test/dog/dog.12111.jpg',
       'test/dog/dog.12022.jpg', 'test/cat/cat.12326.jpg',
       'test/dog/dog.12130.jpg', 'test/cat/cat.12407.jpg',
       'test/dog/dog.12072.jpg', 'test/cat/cat.12414.jpg',
       'test/dog/dog.12100.jpg', 'test/cat/cat.12271.jpg',
       'test/cat/cat.12368.jpg', 'test/cat/cat.12251.jpg',
       'test/cat/cat.12260.jpg', 'test/dog/dog.12002.jpg',
       'test/dog/dog.12116.jpg', 'test/cat/cat.12429.jpg',
       'test/dog/dog.12170.jpg', 'test/cat/cat.12363.jpg',
       'test/dog/dog.12045.jpg', 'test/dog/dog.12141.jpg',
       'test/dog/dog.12096.jpg', 'test/cat/cat.12470.jpg',
       'test/cat/cat.12474.jpg', 'test/cat/cat.12463.jpg',
       'test/dog/dog.12194.jpg', 'test/dog/dog.12127.jpg',
       'test/dog/dog.12060.jpg', 'test/cat/cat.12287.jpg

In [102]:
paths = np.array(data['filenames'])
targets = to_categorical(data['target'])

In [103]:
test_files, test_target = data['filenames'], targets

In [104]:
from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

In [105]:
def path_to_tensor(img_path):
  img= load_img(img_path, target_size=(224,224))
  x = img_to_array(img)
  # return x[np.newaxis, :]
  return np.expand_dims(x, axis=0)

In [106]:
def paths_to_tensor(img_paths):
  list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
  return np.vstack(list_of_tensors)

In [107]:
test_tensor = paths_to_tensor(test_files)

In [108]:
test_tensor=preprocess_input(test_tensor) #vgg16에 맞게 데이터를 preprocessing

In [110]:
model.evaluate(test_tensor, test_target)

# test 의 loss, accuracy

15/15 [==============================] - 3s 148ms/step - loss: 0.2731 - accuracy: 0.8780


[0.2731117904186249, 0.8780487775802612]

-----

-------실제 예측-------

In [111]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/dog-1203760_960_720.jpg', target_size=(224,224))

In [112]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/dog.jpg', target_size=(224,224))

In [113]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/pred001.jfif', target_size=(224,224))

In [114]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/_126241775_getty_cats.png', target_size=(224,224))

In [115]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/cat.jfif', target_size=(224,224))

In [116]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/cat_dogs/main-ad65ae47-5a50-456d-a41f-528b63071b7b.jpg', target_size=(224,224))

JPG 용

In [117]:
num = ['0562', '1311', '1459', '1464', '2229', '2232','0914','0915']
for i in num :
  img = load_img(f'/content/drive/MyDrive/Colab Notebooks/cat_dogs/IMG_{i}.jpg', target_size=(224,224))

  img = img_to_array(img)
  img = img.reshape((1, img.shape[0],img.shape[1],img.shape[2]))
  img = preprocess_input(img)
  pred = new_model.predict(img)
  if np.argmax(pred) == 1:
    print('i', '이거슨 \n개!')
  else :
    print('i', '이거슨 \n고양이!')

1/1 [==============================] - 0s 17ms/step
i 이거슨 
개!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
개!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 18ms/step
i 이거슨 
개!
1/1 [==============================] - 0s 18ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
개!


JPEG용

In [118]:
num = ['0916','0917','0918']
for i in num :
  img = load_img(f'/content/drive/MyDrive/Colab Notebooks/cat_dogs/IMG_{i}.jpeg', target_size=(224,224))

  img = img_to_array(img)
  img = img.reshape((1, img.shape[0],img.shape[1],img.shape[2]))
  img = preprocess_input(img)
  pred = new_model.predict(img)
  if np.argmax(pred) == 1:
    print('i', '이거슨 \n개!')
  else :
    print('i', '이거슨 \n고양이!')

1/1 [==============================] - 0s 16ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
고양이!
1/1 [==============================] - 0s 16ms/step
i 이거슨 
고양이!
